In [ ]:
!git clone https://github.com/ashawkey/stable-dreamfusion.git

In [ ]:
 %pip install trimesh
 %pip install pythreejs

In [ ]:
!sudo apt-get install libeigen3-dev -y


In [ ]:
%cd /notebooks/stable-dreamfusion


In [ ]:
!git pull origin main

In [ ]:
%pip install xformers>=0.0.20 torch==2.0.1+cu118 torchvision  torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
%pip install  -r requirements.txt --force-reinstall
%pip install git+https://github.com/NVlabs/nvdiffrast/ 



In [ ]:
%pip install git+https://github.com/ashawkey/cubvh --force-reinstall

In [ ]:

# install CUDA extensions (takes about 8 minutes!)
%pip install ./raymarching --force-reinstall
%pip install ./shencoder --force-reinstall
%pip install ./freqencoder --force-reinstall
%pip install ./gridencoder --force-reinstall
#%pip install ./raymarching

In [ ]:
%cd /notebooks/stable-dreamfusion

In [ ]:
%env CUDA_VISIBLE_DEVICES=0
workspace ="wyvern2"
image_path = f'/notebooks/threestudio/load/images/{workspace}_rgba.png'
image_path_without_rgba = image_path.replace('_rgba', '')


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image

opt = [
    "path" => image_path_without_rgba,
]

out_dir = os.path.dirname(opt.path)
out_rgba = os.path.join(out_dir, os.path.basename(opt.path).split('.')[0] + '_rgba.png')
out_depth = os.path.join(out_dir, os.path.basename(opt.path).split('.')[0] + '_depth.png')
out_normal = os.path.join(out_dir, os.path.basename(opt.path).split('.')[0] + '_normal.png')
out_caption = os.path.join(out_dir, os.path.basename(opt.path).split('.')[0] + '_caption.txt')

# load image
print(f'[INFO] loading image...')
final_rgba = cv2.imread(opt.path, cv2.IMREAD_UNCHANGED)
if final_rgba.shape[-1] == 4:
    final_rgba = cv2.cvtColor(final_rgba, cv2.COLOR_BGRA2RGB)
else:
    final_rgba = cv2.cvtColor(final_rgba, cv2.COLOR_BGR2RGB)
    
final_depth = cv2.imread(out_depth, cv2.IMREAD_UNCHANGED)
if final_depth.shape[-1] == 4:
    final_depth = cv2.cvtColor(out_depth, cv2.COLOR_BGRA2RGB)
else:
    final_depth = cv2.cvtColor(out_depth, cv2.COLOR_BGR2RGB)

    
final_normal = cv2.imread(out_normal, cv2.IMREAD_UNCHANGED)
if final_normal.shape[-1] == 4:
    final_normal = cv2.cvtColor(out_normal, cv2.COLOR_BGRA2RGB)
else:
    final_normal = cv2.cvtColor(out_normal, cv2.COLOR_BGR2RGB)
# write output
cv2.imwrite(out_rgba, cv2.cvtColor(final_rgba, cv2.COLOR_RGBA2BGRA))
cv2.imwrite(out_depth, final_depth)
cv2.imwrite(out_normal, final_normal)



In [ ]:
!rm -rf ./{workspace}
!rm -rf ./{workspace}_dmtet

In [ ]:
%run preprocess_image.py {image_path_without_rgba}

In [ ]:
batch_size = 13 # 24


In [ ]:

#%run preprocess_image.py front.png
## zero123 train
# pass in the processed <image>_rgba.png by  --default_polar 20 --image and do NOT pass in --text to enable zero-1-to-3 backend.--default_polar 0
%run main.py -O --image {image_path} --workspace {workspace}  --iters 5000 --test_interval 10 --batch_size {batch_size}

In [ ]:
%run main.py -O --batch_size {batch_size} --image {image_path} --test_interval 5 --workspace {workspace}_dmtet --dmtet   --init_with {workspace}/checkpoints/df.pth


In [ ]:
%run main.py -O --image {image_path} --workspace {workspace}_dmtet --dmtet --test --save_mesh 

In [ ]:
%cd /notebooks/stable-dreamfusion/{workspace}_dmtet/
!zip {workspace}.zip -r mesh
%cd ../

OTHER

In [ ]:
%run main.py -O  --image {image_path} --workspace {workspace}_dmtet_mesh  --dmtet --iters 5000 --init_with  {workspace}_dmtet/mesh/mesh.obj --lock_geo

In [ ]:
%run main.py -O --image {image_path} --workspace {workspace}_dmtet_mesh --dmtet --test --save_mesh

SIMPLE EXPORT

In [ ]:
%run main.py -O --image {image_path} --workspace {workspace} --test --save_mesh  

In [ ]:
!chmod +x scripts/run_image_procedure.sh

In [ ]:
import os
import subprocess

def run_image_procedure(GPU_ID, GUIDANCE_INTERVAL, DEFAULT_POLAR, PREFIX, PROMPT):
    EPOCHS1 = 100
    EPOCHS2 = 200
    EPOCHS3 = 300
    IMAGE = f"/notebooks/threestudio/load/images/{PREFIX}.png"
    IMAGE_RGBA = f"/notebooks/threestudio/load/images/{PREFIX}_rgba.png"
    WS_PH1 = f"trial_{PREFIX}-ph1"
    WS_PH2 = f"trial_{PREFIX}-ph2"
    WS_PH3 = f"trial_{PREFIX}-ph3"
    CKPT1 = f"{WS_PH1}/checkpoints/df_ep0{EPOCHS1}.pth"
    CKPT2 = f"{WS_PH2}/checkpoints/df_ep0{EPOCHS2}.pth"
    CKPT3 = f"{WS_PH3}/checkpoints/df_ep0{EPOCHS3}.pth"

    if not os.path.isfile(IMAGE_RGBA):
        %run preprocess_image.py {IMAGE}

    if not os.path.isfile(CKPT1):
        %run main.py -O --image {IMAGE_RGBA} --workspace {WS_PH1} --default_polar {DEFAULT_POLAR} --iters {EPOCHS1}00 --save_guidance --save_guidance_interval {GUIDANCE_INTERVAL} --batch_size 6 --test_interval 2 --h 96 --w 96 --zero123_grad_scale None --lambda_3d_normal_smooth 0 --dont_override_stuff --fovy_range 20 20 --guidance_scale 5 

    GUIDANCE_INTERVAL = 7
    if not os.path.isfile(CKPT2):
        %run main.py -O --image {IMAGE_RGBA} --workspace {WS_PH2} --text "{PROMPT}" --default_polar {DEFAULT_POLAR} --iters {EPOCHS2}00 --ckpt {CKPT1} --save_guidance --save_guidance_interval {GUIDANCE_INTERVAL} --h 128 --w 128 --albedo_iter_ratio 0.0 --t_range 0.2 0.6 --batch_size 4 --radius_range 2.2 2.6 --test_interval 2 --vram_O --guidance_scale 10 --jitter_pose --jitter_center 0.1 --jitter_target 0.1 --jitter_up 0.05 --known_view_noise_scale 0 --lambda_depth 0 --lr 0.003 --progressive_view --progressive_view_init_ratio 0.05 --known_view_interval 2 --dont_override_stuff --lambda_3d_normal_smooth 1 --textureless_ratio 0.0 --min_ambient_ratio 0.3 --exp_start_iter {EPOCHS1}00 --exp_end_iter {EPOCHS2}00

    if not os.path.isfile(CKPT3):
        %run main.py -O --image {IMAGE_RGBA} --workspace {WS_PH3} --text "{PROMPT}" --default_polar {DEFAULT_POLAR} --iters {EPOCHS3}00 --ckpt {CKPT2} --save_guidance --save_guidance_interval {GUIDANCE_INTERVAL} --h 512 --w 512 --albedo_iter_ratio 0.0 --t_range 0.0 0.5 --batch_size 1 --radius_range 3.2 3.6 --test_interval 2 --vram_O --guidance_scale 10 --jitter_pose --jitter_center 0.015 --jitter_target 0.015 --jitter_up 0.05 --known_view_noise_scale 0 --lambda_depth 0 --lr 0.003 --known_view_interval 2 --dont_override_stuff --lambda_3d_normal_smooth 0.5 --textureless_ratio 0.0 --min_ambient_ratio 0.3 --exp_start_iter {EPOCHS2}00 --exp_end_iter {EPOCHS3}00
run_image_procedure(0, 30, 70, "turtle_plant", "an 3D render of a pokemon ((mix between turtle and plant)) with ((red mark on feet)) and ((leaf crown)) on head")


In [ ]:

import os
def delete_workspace(PREFIX):
    WS_PH1 = f"trial_{PREFIX}-ph1"
    WS_PH2 = f"trial_{PREFIX}-ph2"
    WS_PH3 = f"trial_{PREFIX}-ph3"
    
    for workspace in [WS_PH1, WS_PH2, WS_PH3]:
        if os.path.isdir(workspace):
            os.system(f"rm -r {workspace}")
            print(f"Deleted workspace: {workspace}")
        else:
            print(f"No such workspace: {workspace}")
delete_workspace("turtle_plant")